In [1]:
import pandas as pd
import json
import os
import pytz
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from sklearn.ensemble import IsolationForest

In [2]:
df=pd.read_csv('Dataset/cleaned_texas_data.csv')

#Rename a column
df=df.rename(columns={'san antonio': 'san_antonio'})

df.to_csv('Dataset/cleaned_texas_data.csv',index=False)


In [3]:
#Load your CSV file
df=pd.read_csv('Dataset/cleaned_subregion_data.csv')

#Replace spaces with underscores in the 'city' column
df['city']=df['city'].str.replace(' ','_')

#(Optional) Save the updated dataframe to a new CSV file
df.to_csv('Dataset/cleaned_subregion_data.csv',index=False)


In [4]:
folder='Dataset' 
alldata=[]

#Loop through each JSON file
for filename in os.listdir(folder):
    if filename.endswith('.json'):
        cityname=os.path.splitext(filename)[0]  #Remove .json extension
        filepath=os.path.join(folder,filename)
        
        with open(filepath,'r') as f:
            records=json.load(f)  #This is a list of dictionaries
            for record in records:
                record['city']=cityname  #Add city to each record
                alldata.append(record)

#Convert to DataFrame
df=pd.DataFrame(alldata)

#Save to CSV
df.to_csv('combined.csv',index=False)
print("CSV file 'combined.csv' created successfully.")


CSV file 'combined.csv' created successfully.


In [5]:
#Load the combined CSV
df=pd.read_csv('combined.csv')

#Convert UNIX timestamp to datetime
df['utc_time']=pd.to_datetime(df['time'],unit='s',utc=True)

citytimezonemap = {
    'dallas':'America/Chicago',
    'nyc':'America/New_York',
    'houston':'America/Chicago',
    'la':'America/Los_Angeles',
    'philadelphia':'America/New_York',
    'phoenix':'America/Phoenix',
    'san_antonio':'America/Chicago',
    'san_diego':'America/Los_Angeles',
    'san_jose':'America/Los_Angeles',
    'seattle':'America/Los_Angeles'
}

#Convert to local time based on city
def converttolocal(row):
    tz_name=citytimezonemap.get(row['city'].lower(),'UTC')
    local_tz=pytz.timezone(tz_name)
    return row['utc_time'].astimezone(local_tz)

df['local_time']=df.apply(converttolocal,axis=1)

#Keep only the requested columns
dffiltered=df[['local_time','utc_time','city','temperature','humidity','windSpeed']]

#Save to new CSV
dffiltered.to_csv('finalcombined.csv',index=False)
print("Final CSV with time conversion saved as 'finalcombined.csv'.")


Final CSV with time conversion saved as 'finalcombined.csv'.


In [6]:
#PART 1:Merge weather data with first two demand datasets

#Load weather dataset

df=pd.read_csv('finalcombined.csv')

#Convert 'utc_time' to datetime and remove timezone info for consistency
df['utc_time']=pd.to_datetime(df['utc_time']).dt.tz_localize(None)
df['city']=df['city'].str.lower()

#Load and clean first demand dataset (balance data)

balancedf=pd.read_csv('Dataset/cleaned_balance_data.csv')
balancedf['utc_time']=pd.to_datetime(balancedf['utc_time']).dt.tz_localize(None)
balancedf['city']=balancedf['city'].str.lower()
balancedf=balancedf[['city','utc_time','demand']]

#Load and clean second demand dataset (subregion data)

subregiondf=pd.read_csv('Dataset/cleaned_subregion_data.csv')
subregiondf['utc_time']=pd.to_datetime(subregiondf['utc_time']).dt.tz_localize(None)
subregiondf['city']=subregiondf['city'].str.lower()
subregiondf=subregiondf[['city','utc_time','demand']]

#Combine first two demand datasets
combineddemand=pd.concat([balancedf,subregiondf],ignore_index=True)

#Merge weather data with combined demand data
weatherwithdemand=pd.merge(
    df,
    combineddemand,
    on=['city','utc_time'],
    how='left'
)

#PART 2:Add the wide-format demand data

widedemanddf=pd.read_csv('Dataset/cleaned_texas_data.csv')

#Convert from wide to long format
longdemanddf=pd.melt(
    widedemanddf,
    id_vars=['date'],
    value_vars=['houston','san_antonio','dallas'],
    var_name='city',
    value_name='demand'
)

#Convert date to datetime and use it as local_time
longdemanddf['local_time']=pd.to_datetime(longdemanddf['date']).dt.tz_localize(None)
longdemanddf=longdemanddf.drop('date',axis=1)

#Convert local_time to utc_time based on city timezone
#Define timezone offsets (hours from UTC)
timezoneoffsets={
    'houston':-6, #Central Time
    'dallas':-6,  #Central Time
    'san_antonio':-6  #Central Time
}

#Convert local time to UTC based on city timezone
def localtoutc(row):
    offset=timezoneoffsets.get(row['city'],-6)  #Default to Central Time
    return row['local_time'] + pd.Timedelta(hours=offset)

#Apply the conversion
longdemanddf['utc_time']=longdemanddf.apply(localtoutc,axis=1)
longdemanddf['city']=longdemanddf['city'].str.lower()
#print(f"Transformed wide format data shape:{longdemanddf.shape}")

#Merge with the existing weather and demand data
#First,preserve existing demand values
weatherwithdemand['demand_original']=weatherwithdemand['demand']

#Perform the merge
finalmergeddf=pd.merge(
    weatherwithdemand,
    longdemanddf,
    on=['city','utc_time'],
    how='left',
    suffixes=('','_new')
)

#Handle demand values (prioritize new values where available)
#Where new demand exists,use it; otherwise keep original
finalmergeddf['demand']=finalmergeddf['demand_new'].fillna(finalmergeddf['demand_original'])

#Clean up intermediate columns
finalmergeddf=finalmergeddf.drop(['demand_new','demand_original'],axis=1)

#PART 3:Clean and save the final dataset

#Sort by city and time for readability
finalmergeddf=finalmergeddf.sort_values(['city','utc_time'])

#Create a cleaned version with no missing demand values
cleaneddf=finalmergeddf.dropna(subset=['demand'])
cleaneddf.to_csv('finaldataset.csv',index=False)

#Load the CSV file
df=pd.read_csv('finaldataset.csv')

#Drop a column (e.g.,'city_code')
df=df.drop(columns=['local_time_new'])

#Save the updated DataFrame back to CSV
df.to_csv('finaldataset.csv',index=False)
print("✅ Cleaned dataset saved as 'finaldataset.csv'")



C:\Users\riyya\AppData\Local\Temp\ipykernel_1688\336661894.py:101: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('finaldataset.csv')


✅ Cleaned dataset saved as 'finaldataset.csv'


In [7]:
#Load CSV without date parsing first
df=pd.read_csv("finaldataset.csv")

#Convert local_time to datetime and remove timezone
df["local_time"]=pd.to_datetime(df["local_time"],utc=True).dt.tz_localize(None)

#Now safe to use .dt accessors
df["hour"]=df["local_time"].dt.hour
df["dayofweek"]=df["local_time"].dt.dayofweek
df["month"]=df["local_time"].dt.month


#Define seasons (Northern Hemisphere)
def get_season(month):
    if month in [12,1,2]:
        return 'Winter'
    elif month in [3,4,5]:
        return 'Spring'
    elif month in [6,7,8]:
        return 'Summer'
    else:
        return 'Fall'

df["season"]=df["month"].apply(get_season)

#Normalize continuous features
scaler=StandardScaler()
df[["temperature","humidity","windSpeed","demand"]]=scaler.fit_transform(
    df[["temperature","humidity","windSpeed","demand"]])

#Aggregation

#Daily summary stats
dailysummary=df.resample('D',on='local_time').agg({
    "temperature": ["mean","min","max"],
    "humidity": ["mean"],
    "windSpeed": ["mean"],
    "demand": ["sum","mean","max"]
}).reset_index()

#Weekly summary stats
weeklysummary=df.resample('W',on='local_time').agg({
    "temperature": ["mean","min","max"],
    "humidity": ["mean"],
    "windSpeed": ["mean"],
    "demand": ["sum","mean","max"]
}).reset_index()


print("\n--- Daily Summary Statistics ---")
print(dailysummary.head())

print("\n--- Weekly Summary Statistics ---")
print(weeklysummary.head())

#Anomaly & Error Detection

#Z-Score method
df["demand_z"]=stats.zscore(df["demand"])
zanomalies=df[np.abs(df["demand_z"]) > 3]

#Isolation Forest
features=scaler.fit_transform(df[["temperature","humidity","windSpeed","demand"]])
isoforest=IsolationForest(contamination=0.01,random_state=42)
df["iforest_flag"]=isoforest.fit_predict(features)
iforestanomalies=df[df["iforest_flag"] == -1]

#Display
print("\n--- Z-Score Based Demand Anomalies ---")
print(zanomalies[["local_time","demand","demand_z"]].head())

print("\n--- Isolation Forest Detected Anomalies ---")
print(iforestanomalies[["local_time","temperature","humidity","windSpeed","demand"]].head())


#Save processed data


df=df.drop('iforest_flag',axis=1)
df=df.drop('demand_z',axis=1)
#Drop rows with any missing values
df=df.dropna(subset=['temperature','humidity','windSpeed','demand'])

df.to_csv("scaleddataset.csv",index=False)



--- Daily Summary Statistics ---
  local_time temperature                      humidity windSpeed     demand  \
                    mean       min       max      mean      mean        sum   
0 2018-07-01    0.466620 -0.353181  2.362830 -0.235490 -0.418424 -25.841166   
1 2018-07-02    0.924849 -0.612149  2.618640 -0.263539 -0.212833  80.988616   
2 2018-07-03    0.993066 -0.783321  2.690015 -0.238172 -0.129051  96.370379   
3 2018-07-04    0.904790 -0.459295  2.633800 -0.100643 -0.191089  40.606722   
4 2018-07-05    0.960318 -0.326021  3.062677 -0.007361 -0.261514  70.559797   

                       
       mean       max  
0 -0.323015  1.520864  
1  0.373219  3.608134  
2  0.401543  3.607970  
3  0.169195  2.806371  
4  0.293999  3.266932  

--- Weekly Summary Statistics ---
  local_time temperature                      humidity windSpeed      demand  \
                    mean       min       max      mean      mean         sum   
0 2018-07-01    0.466620 -0.353181  2.362830 -0.2